In [1]:
import os

In [2]:
%pwd

'c:\\SkinDiseaseProject\\skin_disease_predictor\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\SkinDiseaseProject\\skin_disease_predictor'

In [5]:
import tensorflow as tf

In [9]:
from tensorflow.keras.metrics import Metric
class F1Score(Metric):
    def __init__(self, name='f1_score', dtype=None):
        super(F1Score, self).__init__(name=name, dtype=dtype)
        self.true_positive = self.add_weight(name="tp", initializer="zeros")
        self.false_positive = self.add_weight(name="fp", initializer="zeros")
        self.false_negative = self.add_weight(name="fn", initializer="zeros")

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.round(y_pred)  # Convert predictions to binary (0 or 1)
        tp = tf.count_nonzero(y_true * y_pred)
        fp = tf.count_nonzero((y_pred) * (y_true - 1))
        fn = tf.count_nonzero((y_true) * (y_pred - 1))

        self.true_positive.assign_add(tp)
        self.false_positive.assign_add(fp)
        self.false_negative.assign_add(fn)

    def result(self):
        precision = self.true_positive / (self.true_positive + self.false_positive)
        recall = self.true_positive / (self.true_positive + self.false_negative)
        return 2 * (precision * recall) / (precision + recall)

    def reset_state(self):
        self.true_positive.assign(0)
        self.false_positive.assign(0)
        self.false_negative.assign(0)

In [10]:
model = tf.keras.models.load_model('artifacts/training/model.keras',custom_objects={"F1Score": F1Score})

In [12]:
from dataclasses import dataclass
from pathlib import Path
 
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params : dict
    params_image_size: int
    params_batch_size: int

In [13]:
from Acne_Prediction.constants import *
from Acne_Prediction.utils.common import read_yaml,save_json, create_directories

In [14]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model= "artifacts/training/model.keras",
            training_data = "artifacts/data_ingestion/Dataset_new",
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )
        return eval_config

In [15]:
class Evaluation:
    def __init__(self,config: EvaluationConfig):
        self.config = config
        
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split = 0.3
        )
    
        dataflow_kwargs = dict(
        target_size = self.config.params_image_size[:-1],
        batch_size = self.config.params_batch_size,
        class_mode = 'categorical'
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self._valid_generator = valid_datagenerator.flow_from_directory(
            directory = self.config.training_data,
            subset = 'validation',
            shuffle = False,
            **dataflow_kwargs
        )
    
    @staticmethod
    def load_model(model_path: str) -> tf.keras.models.Model:
        return tf.keras.models.load_model(model_path, custom_objects={"F1Score": F1Score})
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self._valid_generator)

    def save_score(self):
        scores = {"loss":self.score[0], "accuracy":self.score[1]}
        save_json(path = Path("scores.json"), data=scores)


In [ ]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
    raise e